# 1. Install and Import Dependencies

In [ ]:
!pip install tensorflow tensorflow-gpu opencv-python matplotlib.pyplot

In [1]:
import tensorflow as tf
import os

# 2. Remove unfit images

In [2]:
import cv2
import imghdr

In [3]:
data_dir = 'cats and dogs-train' # define data directory

In [4]:
image_exts = ['jpeg','jpg', 'bmp', 'png'] # required image extensions

In [ ]:
# Loop through each class in the dataset directory
for image_class in os.listdir(data_dir):  
    # Loop through each image in the current class directory
    for image in os.listdir(os.path.join(data_dir, image_class)):  
        # Construct the full file path for the image
        image_path = os.path.join(data_dir, image_class, image)  
        try:
            # Attempt to read the image using OpenCV
            img = cv2.imread(image_path)  
            # Check the image's file extension/type using imghdr
            tip = imghdr.what(image_path)  
            
            # If the image's extension/type is not in the list of valid extensions
            if tip not in image_exts:  
                # Print a message indicating the image is invalid
                print('Image not in ext list {}'.format(image_path))  
                # Remove the invalid image from the directory
                os.remove(image_path)  
        except Exception as e:
            # If an exception occurs (e.g., corrupted image), print the issue
            print('Issue with image {}'.format(image_path))  
            # os.remove(image_path)


# 3. Load Data

In [5]:
import numpy as np
from matplotlib import pyplot as plt

In [6]:
# Load images from the specified directory and its subdirectories
# 'data_dir' is the path to the folder where images are organized in subdirectories by class
data = tf.keras.utils.image_dataset_from_directory(data_dir, batch_size=32)

# The function automatically:
# - Loads all the images from the 'data_dir'
# - Assigns labels based on the subdirectory names (each subdirectory represents a class)
# - Returns a 'tf.data.Dataset' object that can be used to efficiently handle the image data


Found 557 files belonging to 2 classes.


In [7]:
data_iterator = data.as_numpy_iterator() #convert data to numpy iterator

In [8]:
# Retrieve the next batch of data (images and labels) from the dataset using the NumPy iterator
batch = data_iterator.next()

# The batch contains a tuple with two elements:
# - The first element is a batch of images (as NumPy arrays)
# - The second element is the corresponding labels for those images


In [9]:
batch[0].shape

(32, 256, 256, 3)

In [10]:
import gc
gc.collect()


0

In [11]:
batch[0][8]

array([[[172.05078 , 171.07642 , 139.99951 ],
        [172.86328 , 171.9231  , 140.74365 ],
        [175.99146 , 175.99146 , 139.99146 ],
        ...,
        [116.86328 , 105.86328 ,  99.86328 ],
        [113.940186, 107.940186,  95.940186],
        [112.      , 106.      ,  94.      ]],

       [[168.      , 169.      , 128.17969 ],
        [167.17969 , 168.17969 , 127.359375],
        [167.58984 , 167.58984 , 131.58984 ],
        ...,
        [130.85962 , 117.85962 , 109.833984],
        [111.58984 , 105.58984 ,  93.58984 ],
        [111.58984 , 105.58984 ,  93.58984 ]],

       [[168.      , 168.      , 130.      ],
        [164.13843 , 164.13843 , 126.13843 ],
        [165.      , 165.      , 129.      ],
        ...,
        [156.99219 , 142.99219 , 131.99219 ],
        [115.875   , 109.875   ,  97.875   ],
        [111.      , 105.      ,  93.      ]],

       ...,

       [[ 61.      ,  70.      ,  49.      ],
        [ 62.17798 ,  71.17798 ,  50.17798 ],
        [ 60.316406,  

# 4. Scale Data

In [12]:
data = data.map(lambda x,y: (x/255, y)) # scale data to min 0, max 1

In [13]:
batch = data.as_numpy_iterator().next()
batch

(array([[[[0.52438724, 0.45772058, 0.38713235],
          [0.56855136, 0.5018847 , 0.43129644],
          [0.5740196 , 0.50735295, 0.43676472],
          ...,
          [0.56976104, 0.50701594, 0.44427082],
          [0.55597425, 0.49322918, 0.43048406],
          [0.55003065, 0.48728552, 0.42454043]],
 
         [[0.5807598 , 0.51409316, 0.4435049 ],
          [0.622886  , 0.55621934, 0.48563114],
          [0.6276348 , 0.56096816, 0.4903799 ],
          ...,
          [0.56976104, 0.50701594, 0.44427082],
          [0.55597425, 0.49322918, 0.43048406],
          [0.55003065, 0.48728552, 0.42454043]],
 
         [[0.61219364, 0.545527  , 0.47493872],
          [0.6543199 , 0.58765316, 0.5170649 ],
          [0.6648682 , 0.5982015 , 0.5276133 ],
          ...,
          [0.56976104, 0.50701594, 0.44427082],
          [0.55597425, 0.49322918, 0.43048406],
          [0.55003065, 0.48728552, 0.42454043]],
 
         ...,
 
         [[0.5967132 , 0.4629246 , 0.29486635],
          [0.64192

# 5. Split Data

In [14]:
len(data)

18

In [15]:
train_size = int(len(data)*.8)
val_size = int(len(data)*.2)+1

In [16]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)

# 6. Build Deep Learning Model

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [18]:
model = Sequential()

In [19]:
# Add a 2D Convolutional layer with 16 filters, a 3x3 kernel, stride of 1, and ReLU activation
# - input_shape: (256, 256, 3) specifies the shape of the input image (256x256 pixels with 3 channels for RGB)
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))

# Add a MaxPooling layer to reduce the spatial dimensions (downsampling)
# - Pooling reduces the size of the feature maps, helping to reduce computation and prevent overfitting
model.add(MaxPooling2D())

# Add another Conv2D layer with 32 filters, 3x3 kernel, stride of 1, and ReLU activation
# - This layer learns more complex features (such as edges and textures) from the downsampled feature maps
model.add(Conv2D(32, (3,3), 1, activation='relu'))

# Add another MaxPooling layer to further reduce the size of the feature maps
model.add(MaxPooling2D())

# Add another Conv2D layer with 16 filters, 3x3 kernel, stride of 1, and ReLU activation
# - This layer extracts more specific patterns from the features learned in the previous layers
model.add(Conv2D(16, (3,3), 1, activation='relu'))

# Add another MaxPooling layer to further reduce the spatial dimensions
model.add(MaxPooling2D())

# Flatten the 2D feature maps into a 1D vector to prepare for fully connected layers
# - This is required before passing the data to the Dense layers
model.add(Flatten())

# Add a Dense (fully connected) layer with 256 units and ReLU activation
# - This layer learns to combine the features extracted by the Conv2D layers
model.add(Dense(256, activation='relu'))

# Add the final output Dense layer with 1 unit and sigmoid activation
# - Sigmoid activation is used for binary classification (output is between 0 and 1)
model.add(Dense(1, activation='sigmoid'))


In [20]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy']) # compile model

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 16)        4624      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 30, 30, 16)       0

# 7. Train

In [22]:
#logdir='logs'

In [23]:
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir) # create a tensorboard callback to log metrics

In [24]:
hist = model.fit(train, epochs=20, validation_data=val) # fit model to train data

Epoch 1/20
14/14 [==============================] - 17s 1s/step - loss: 0.7728 - accuracy: 0.5357 - val_loss: 0.6875 - val_accuracy: 0.5872
Epoch 2/20
14/14 [==============================] - 15s 984ms/step - loss: 0.6889 - accuracy: 0.5402 - val_loss: 0.6923 - val_accuracy: 0.4954
Epoch 3/20
14/14 [==============================] - 25s 2s/step - loss: 0.6905 - accuracy: 0.5223 - val_loss: 0.6871 - val_accuracy: 0.4862
Epoch 4/20
14/14 [==============================] - 20s 1s/step - loss: 0.6803 - accuracy: 0.5536 - val_loss: 0.7353 - val_accuracy: 0.5046
Epoch 5/20
14/14 [==============================] - 20s 1s/step - loss: 0.6655 - accuracy: 0.6272 - val_loss: 0.6277 - val_accuracy: 0.7156
Epoch 6/20
14/14 [==============================] - 19s 1s/step - loss: 0.5793 - accuracy: 0.7143 - val_loss: 0.5106 - val_accuracy: 0.7890
Epoch 7/20
14/14 [==============================] - 18s 1s/step - loss: 0.4630 - accuracy: 0.7969 - val_loss: 0.3481 - val_accuracy: 0.8716
Epoch 8/20
14/14 

# 8. Plot Performance

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# Test data

In [25]:
data_dir_test = 'cats and dogs-test' # define data directory

In [26]:
# Load images from the specified directory and its subdirectories
# 'data_dir' is the path to the folder where images are organized in subdirectories by class
data_test = tf.keras.utils.image_dataset_from_directory(data_dir_test, batch_size=32) 

Found 140 files belonging to 2 classes.


# 9. Evaluate

In [27]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [28]:
precision = Precision()
recall = Recall()
accuracy = BinaryAccuracy()

In [29]:
for batch in data_test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    precision.update_state(y, yhat)
    recall.update_state(y, yhat)
    accuracy.update_state(y, yhat)

1/1 [==============================] - 0s 205ms/step


In [30]:
final_precision = precision.result().numpy()
final_recall = recall.result().numpy()
final_accuracy = accuracy.result().numpy()

# Print the accumulated metrics for the entire dataset
print(f'Precision: {final_precision}')
print(f'Recall: {final_recall}')
print(f'Accuracy: {final_accuracy}')

Precision: 0.6351351141929626
Recall: 0.6714285612106323
Accuracy: 0.6428571343421936


# 10. Make test prediction

In [31]:
import cv2

In [32]:
img = cv2.imread('dog_11.jpg')
#plt.imshow(img)
#plt.show()

In [33]:
resize = tf.image.resize(img, (256,256))
#plt.imshow(resize.numpy().astype(int))
#plt.show()

In [34]:
yhat = model.predict(np.expand_dims(resize/255, 0))
yhat

1/1 [==============================] - 0s 40ms/step


array([[0.99342996]], dtype=float32)

In [35]:
if yhat > 0.5: 
    print(f'Predicted class is Dog')
else:
    print(f'Predicted class is Cat')

Predicted class is Dog


# 11. Save the Model

In [36]:
from tensorflow.keras.models import load_model

In [37]:
model.save(os.path.join('models','classifier.h5'))

In [39]:
new_model = load_model('models/classifier.h5')

In [42]:
yhat_new = new_model.predict(np.expand_dims(resize/255, 0))

1/1 [==============================] - 0s 36ms/step


In [43]:
if yhat_new > 0.5: 
    print(f'Predicted class is Dog')
else:
    print(f'Predicted class is Cat')

Predicted class is Dog
